In [4]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt

path = "Duke Ellington - It don_t mean a thing (1943).m4a"

# Choose analysis sample rate and channel count
fs = 44100
channels = 1

# Decode to raw float32 mono PCM via ffmpeg -> stdout
cmd = [
    "ffmpeg",
    "-hide_banner",
    "-loglevel",
    "error",
    "-i",
    path,
    "-ac",
    str(channels),
    "-ar",
    str(fs),
    "-f",
    "f32le",
    "pipe:1",
]

proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
x = np.frombuffer(proc.stdout, dtype=np.float32)

# Optional: analyze only first N seconds
seconds = 10
N = min(len(x), int(seconds * fs))
x = x[:N]

# Window + FFT
w = np.hanning(len(x))
X = np.fft.rfft(x * w)
f = np.fft.rfftfreq(len(x), d=1.0 / fs)

mag_db = 20 * np.log10(np.abs(X) + 1e-12)

%matplotlib qt
plt.figure()
plt.plot(f, mag_db)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude (dB)")
plt.title(f"FFT magnitude (first {seconds}s, fs={fs} Hz)")
plt.xlim(0, min(20000, fs / 2))
plt.grid(True, ls=":")
plt.show()

In [5]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

path = "Duke Ellington - It don_t mean a thing (1943).m4a"

fs = 44100
channels = 1

cmd = [
    "ffmpeg",
    "-hide_banner",
    "-loglevel",
    "error",
    "-i",
    path,
    "-ac",
    str(channels),
    "-ar",
    str(fs),
    "-f",
    "f32le",
    "pipe:1",
]
proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
x = np.frombuffer(proc.stdout, dtype=np.float32)

# Use a long chunk so low frequencies are well resolved
seconds = 120  # increase to 300, 600, ...
N = min(len(x), int(seconds * fs))
x = x[:N]

# Welch: frequency resolution ~ fs/nperseg, so increase nperseg for finer low-f bins
nperseg = fs * 20  # 20 s segments -> Δf ~ 0.05 Hz
noverlap = int(nperseg * 0.75)

f, Pxx = signal.welch(
    x,
    fs=fs,
    window="hann",
    nperseg=nperseg,
    noverlap=noverlap,
    detrend="constant",
    scaling="density",
)

Pxx_db = 10 * np.log10(Pxx + 1e-30)

plt.figure()
plt.plot(f, Pxx_db)
plt.xlabel("Frequency (Hz)")
plt.ylabel("PSD (dB/Hz)")
plt.title(f"Welch PSD (seconds={seconds}, nperseg={nperseg/fs:.0f}s)")
plt.xlim(0, 500)  # focus low frequencies
plt.grid(True, ls=":")
plt.show()

In [11]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

path = "Duke Ellington - It don_t mean a thing (1943).m4a"

# --- decode to float32 mono at full rate ---
fs = 44100
cmd = [
    "ffmpeg",
    "-hide_banner",
    "-loglevel",
    "error",
    "-i",
    path,
    "-ac",
    "1",
    "-ar",
    str(fs),
    "-f",
    "f32le",
    "pipe:1",
]
proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
x = np.frombuffer(proc.stdout, dtype=np.float32)

# Use a long chunk (resolution ~ 1/T). Start with 3–5 minutes if you have it.
seconds = 180
x = x[: min(len(x), int(seconds * fs))]

# --- envelope extraction (this is the key trick) ---
# Band-limit first so the envelope represents audible energy, not DC/rumble
b, a = signal.butter(4, [30, 8000], btype="bandpass", fs=fs)
xf = signal.filtfilt(b, a, x)

# Hilbert envelope
env = np.abs(signal.hilbert(xf))

# Optional: smooth envelope slightly (e.g. 50 ms)
smooth_s = 0.05
L = max(1, int(smooth_s * fs))
env = signal.filtfilt(np.ones(L) / L, [1.0], env)

# --- downsample envelope to a low rate (keeps only low-frequency modulation) ---
fs_env = 200  # plenty for a few Hz
decim = fs // fs_env
env_ds = signal.decimate(env, decim, ftype="fir", zero_phase=True)
env_ds = env_ds - np.mean(env_ds)  # remove DC

# Optional: high-pass at 0.2 Hz to remove slow drift
bh, ah = signal.butter(2, 0.2, btype="highpass", fs=fs_env)
env_ds = signal.filtfilt(bh, ah, env_ds)

# --- Welch PSD with long segments for fine resolution ---
# Δf ~ fs_env / nperseg. If nperseg = 60 s -> ~0.0167 Hz bins at 200 Hz.
seg_s = 60
nperseg = int(seg_s * fs_env)
noverlap = int(0.75 * nperseg)

f, Pxx = signal.welch(
    env_ds,
    fs=fs_env,
    window="hann",
    nperseg=nperseg,
    noverlap=noverlap,
    detrend="constant",
    scaling="density",
)
Pxx_db = 10 * np.log10(Pxx + 1e-30)

# --- plot low frequencies and mark ~2 Hz ---
plt.figure()
plt.plot(f, Pxx_db)
plt.xlim(0, 10)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Envelope PSD (dB/Hz)")
plt.grid(True, ls=":")
plt.axvline(2.0, linestyle="--")
plt.show()

# Optional: peak picking near 2 Hz
band = (f > 1.5) & (f < 2.5)
i = np.argmax(Pxx[band])
f_peak = f[band][i]
print("Peak near 2 Hz:", f_peak, "Hz")

Peak near 2 Hz: 1.7166666666666666 Hz


In [13]:
main_freq_Hz = 3.42
achtel_Hz = 2 * main_freq_Hz

In [16]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

path = "Duke Ellington - It don_t mean a thing (1943).m4a"

# --- decode audio to float32 mono ---
fs = 44100
cmd = [
    "ffmpeg",
    "-hide_banner",
    "-loglevel",
    "error",
    "-i",
    path,
    "-ac",
    "1",
    "-ar",
    str(fs),
    "-f",
    "f32le",
    "pipe:1",
]
proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
x = np.frombuffer(proc.stdout, dtype=np.float32)

# analyze a longer chunk (resolution/time stability)
seconds = 360
x = x[: min(len(x), int(seconds * fs))]

# --- (A) envelope extraction (recommended for f0 ~ few Hz) ---
# Band-limit audio energy a bit (optional but usually helpful)
b, a = signal.butter(4, [30, 8000], btype="bandpass", fs=fs)
xf = signal.filtfilt(b, a, x)

env = np.abs(signal.hilbert(xf))  # amplitude envelope

# light smoothing (~50 ms)
L = max(1, int(0.05 * fs))
env = signal.filtfilt(np.ones(L) / L, [1.0], env)

# --- (B) downsample envelope to low rate ---
fs_env = 200  # Hz
decim = fs // fs_env
env_ds = signal.decimate(env, decim, ftype="fir", zero_phase=True)
t = np.arange(len(env_ds)) / fs_env

# remove slow drift
bh, ah = signal.butter(2, 0.2, btype="highpass", fs=fs_env)
y = signal.filtfilt(bh, ah, env_ds)
y = y - np.mean(y)

# --- (C) lock-in at f0 ---
f0 = main_freq_Hz  # Hz (set your found frequency)

ref_c = np.cos(2 * np.pi * f0 * t)
ref_s = np.sin(2 * np.pi * f0 * t)

mix_I = y * ref_c
mix_Q = y * ref_s

# Low-pass sets measurement bandwidth / time constant:
# choose lp_hz small enough to average noise but large enough to track changes.
lp_hz = 0.3  # Hz  (roughly sets response time ~ 1/(2π lp_hz))
blp, alp = signal.butter(4, lp_hz, btype="lowpass", fs=fs_env)
I = signal.filtfilt(blp, alp, mix_I)
Q = signal.filtfilt(blp, alp, mix_Q)

R = np.sqrt(I**2 + Q**2)
phi = np.unwrap(np.arctan2(Q, I))

# --- plots ---
plt.figure()
plt.plot(t, y, label="envelope (detrended)")
plt.xlim(0, min(t[-1], 60))
plt.grid(True, ls=":")
plt.legend()
plt.show()

plt.figure()
plt.plot(t, R)
plt.xlabel("Time (s)")
plt.ylabel("Lock-in amplitude R")
plt.grid(True, ls=":")
plt.show()

plt.figure()
plt.plot(t, phi)
plt.xlabel("Time (s)")
plt.ylabel("Lock-in phase phi (rad)")
plt.grid(True, ls=":")
plt.show()

# Phase histogram (to see two phase states)
plt.figure()
plt.hist((phi % (2 * np.pi)), bins=60)
plt.xlabel("phi mod 2π (rad)")
plt.ylabel("count")
plt.show()

In [ ]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# ----------------------------
# 1) Load audio (.m4a) via ffmpeg
# ----------------------------
path = "Duke Ellington - It don_t mean a thing (1943).m4a"

fs_audio = 44100
cmd = [
    "ffmpeg",
    "-hide_banner",
    "-loglevel",
    "error",
    "-i",
    path,
    "-ac",
    "1",
    "-ar",
    str(fs_audio),
    "-f",
    "f32le",
    "pipe:1",
]
proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
x = np.frombuffer(proc.stdout, dtype=np.float32)

seconds = 180  # länger => stabilere Phase
x = x[: min(len(x), int(seconds * fs_audio))]

# ----------------------------
# 2) Envelope (beat/modulation lives here for few-Hz features)
# ----------------------------
# band-limit audio energy (optional but helpful)
b, a = signal.butter(4, [30, 8000], btype="bandpass", fs=fs_audio)
xf = signal.filtfilt(b, a, x)

env = np.abs(signal.hilbert(xf))

# smooth envelope a bit (50 ms)
L = max(1, int(0.05 * fs_audio))
env = signal.filtfilt(np.ones(L) / L, [1.0], env)

# downsample envelope
fs = 200  # target sampling rate for lock-in
decim = fs_audio // fs
env_ds = signal.decimate(env, decim, ftype="fir", zero_phase=True)

# remove drift (important for phase stability)
bh, ah = signal.butter(2, 0.2, btype="highpass", fs=fs)
y = signal.filtfilt(bh, ah, env_ds)
y = y - np.mean(y)

t = np.arange(len(y)) / fs

# ----------------------------
# 3) Frequency sweep lock-in: 6.2..7.0 Hz in 200 steps
# ----------------------------
f_start, f_stop, n_f = 6.2, 7.0, 200
freqs = np.linspace(f_start, f_stop, n_f)

# complex reference matrix: shape (n_f, n_t)
# Z_mix(f,t) = y(t) * exp(-i 2π f t)
phase_mat = -2.0 * np.pi * freqs[:, None] * t[None, :]
ref = np.exp(1j * phase_mat)
z_mix = y[None, :] * ref  # complex

# low-pass bandwidth (lock-in time constant)
lp_hz = 0.3  # smaller => more averaging, larger => more tracking
sos = signal.butter(4, lp_hz, btype="lowpass", fs=fs, output="sos")

# apply same low-pass to real and imag along time axis
z_lp_re = signal.sosfiltfilt(sos, z_mix.real, axis=1)
z_lp_im = signal.sosfiltfilt(sos, z_mix.imag, axis=1)
z_lp = z_lp_re + 1j * z_lp_im

# avoid filter edge transients by discarding a margin
discard_s = 5.0
k0 = int(discard_s * fs)
k1 = len(t) - k0
z_ss = z_lp[:, k0:k1]

# time-average complex lock-in output per frequency
Z = np.mean(z_ss, axis=1)  # shape (n_f,)

amp = np.abs(Z)
phi = np.angle(Z)  # in [-pi, pi]
# optional: unwrap across frequency
phi_unwrap = np.unwrap(phi)

# ----------------------------
# 4) Plot: phase vs frequency (2D line)
# ----------------------------
plt.figure()
plt.plot(freqs, phi_unwrap, marker=".")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Lock-in phase (rad) (unwrapped)")
plt.grid(True, ls=":")
plt.show()

# Optional: amplitude vs frequency
plt.figure()
plt.plot(freqs, amp, marker=".")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Lock-in amplitude")
plt.grid(True, ls=":")
plt.show()

# ----------------------------
# 5) Optional: truly 2D heatmap phase(time, frequency)
# ----------------------------
# This shows if phase is stable or drifts with time
phi_tf = np.angle(z_lp)  # shape (n_f, n_t)
plt.figure()
plt.imshow(
    phi_tf, aspect="auto", origin="lower", extent=[t[0], t[-1], freqs[0], freqs[-1]]
)
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.title("Lock-in phase map (rad)")
plt.colorbar(label="phase (rad)")
plt.show()

: 